# A1 - Doctor Strange

## Abstract

*Implementing BFS, IDS, A star, weighted A star search algorithms to reach the goal in doctor strange game defined in project*

## State properties

each state has these properties:
* doctors positions list
* picked up spells position list
* picked up pills position list
* parent state
* cost of the state
* uniqueness of a state depends on first three properties mentioned above

## Modeling the problem

* initial state: 
    * doctors = [(0, 0)] 
    * spells = []          
    * pills = []            
    * parent = None        
    * cost = 0             

* goal state:
    * all doctors in position (n-1, m-1)
    * all spells are picked up

* actions: 
    * up
    * down
    * right 
    * left

* transition model:
    * up -> (x-1, y)
    * down -> (x+1, y)
    * right -> (x, y+1)
    * left -> (x, y-1)

* path cost:
    * sum of total steps each doctor took to reach the goal

## Take Input

changed input origin coordinations to achieve well-defined definition of matrix

In [49]:
def get_pair_input():
    a, b = map(int, input().strip().split())
    return a, b

n, m = get_pair_input()
c, k = get_pair_input()
grid = [[0 for i in range(m)] for j in range(n)]
row = n-1
all_spells = []
for i in range(c):
    x, y = get_pair_input()
    grid[row-x][y] = 1      # add spells
    all_spells.append((row-x, y))

for i in range(k):
    x, y = get_pair_input()
    grid[row-x][y] = 2      # add pills

d = int(input())
for i in range(d):
    x, y = get_pair_input()
    grid[row-x][y] = 3      # add walls

import numpy as np 
print(np.matrix(grid))
print()

[[0 0 0 0]
 [1 0 0 0]
 [0 0 3 0]
 [0 2 1 0]]



## Define State class

In [50]:
class State:
	def __init__(self, doctors, spells=[], pills=[], cost=0, parent=None):
		self.doctors = doctors
		self.spells = spells
		self.pills = pills 
		self.cost = cost
		self.parent = parent 

	def __eq__(self, object):
		if(isinstance(object, State)):
			return self.spells == object.spells and \
				self.pills == object.pills and \
				self.doctors == object.doctors

	def __str__(self):
		return f" doctors: {self.doctors}\n spells: {self.spells}\n pills: {self.pills}\n cost: {self.cost}\n"

	def __hash__(self) -> int:
		return hash(str(self))

## Define wrapper for execution time of a function

In [51]:
import time 
def execution_time_wrapper(func):
    def inner(*args, **kwargs):
        tic = time.time()

        result, explored_states = func(*args, **kwargs)

        toc = time.time()
        print(f'Compute time: {round(1000 * (toc - tic))}ms')
        return result, explored_states

    return inner

## Print path function

In [52]:
def print_path(goal_state):
    print("path from source to goal: ")
    while goal_state:
        print(goal_state)
        goal_state = goal_state.parent

## Breadth-First Search

In [53]:
from collections import deque as queue


x_dir = [ -1, 0, 1, 0]
y_dir = [ 0, 1, 0, -1]

explored = set()


def is_position_valid(x, y):
    """ check whether new position is valid"""
    if (x < 0 or y < 0 or x >= n or y >= m or grid[x][y] == 3):
        return False

    return True

def apply_event(state, grid, x, y):
    """ pick up consumables such as pills and spells """
    if(grid[x][y] == 1):
        if not (x, y) in state.spells:
            state.spells.append((x, y))

    elif(grid[x][y] == 2):
        if not (x,y) in state.pills:
            state.pills.append((x, y))
            state.doctors.append((0, 0))

def create_child_state(prev_state, x, y, j):
    """" generate child of the state """
    # create new state
    state = State(doctors=prev_state.doctors[:],
    spells=prev_state.spells[:], pills=prev_state.pills[:], 
    cost=prev_state.cost+1, parent=prev_state)

    # update doctor location
    state.doctors[j] = (x, y)

    apply_event(state, grid, x, y)
    
    return state

def goal_test(state):
    """" check if given state reached the goal """
    if len(state.spells) != c:
        return False 

    for doctor in state.doctors:
        if doctor != (0, m-1):
            return False

    return True 
    
def get_neighbors(doctors, j):
    """ get neighbors of a state in the game grid """
    neighbors = []
    for i in range(4):
        adjx = doctors[j][0] + x_dir[i]
        adjy = doctors[j][1] + y_dir[i]
        neighbors.append((adjx, adjy))
    return neighbors


@execution_time_wrapper
def BFS(state):
	
    frontier = queue()
    frontier.append(state)
    explored.add(start_state)
    apply_event(state, grid, n-1, 0)

    while (len(frontier) > 0):
        state = frontier.popleft()
        
        doctors = state.doctors
        for j in range(len(doctors)):
            neighbors = get_neighbors(doctors, j)
            for adjx, adjy in neighbors:
                if (is_position_valid(adjx, adjy)):
                    child_state = create_child_state(state, adjx, adjy, j)
                    if(child_state not in explored):
                        frontier.append(child_state)
                        explored.add(child_state)
                        if goal_test(child_state):
                            return child_state, len(explored)


start_state = State(doctors=[(n-1, 0)])



In [54]:
goal_state, explored_states = BFS(start_state)
print("Unique States Seen: ", explored_states)
print_path(goal_state)

Compute time: 46ms
Unique States Seen:  2456
path from source to goal: 
 doctors: [(0, 3), (0, 3)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 11

 doctors: [(0, 3), (0, 2)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 10

 doctors: [(0, 3), (0, 1)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 9

 doctors: [(0, 3), (0, 0)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 8

 doctors: [(0, 3), (1, 0)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 7

 doctors: [(0, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 6

 doctors: [(1, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 5

 doctors: [(2, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 4

 doctors: [(3, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 3

 doctors: [(3, 2), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 2

 doctors: [(3, 1), (0, 0)]
 spells: []
 pills: [(3, 1)]
 cost: 1

 doctors: [(3, 0)]
 spells: []
 pills: []
 cost: 0



<table>
  <tr>
    <th></th>
    <th>Cost</th>
    <th>Unique States Seen</th>
    <th>Avg exec time (s)</th>
  </tr>
  <tr>
    <td>Test 1</td>
    <td>11</td>
    <td>2456</td>
    <td>0.045</td>
  </tr>
  <tr>
    <td>Test 2</td>
    <td>11</td>
    <td>351141</td>
    <td>10.2</td>
  </tr>
  <tr>
    <td>Test 3</td>
    <td>19</td>
    <td>580226</td>
    <td>13.3</td>
  </tr>
</table>

* Advantages:
    * will find the optimal solution

* Disadvatages:
    * space complexity is exponential
    * uninformed algorithm thus slower than A*

## Iterative-Depth Search

In [55]:
import time 
def execution_time_wrapper(func):
    def inner(*args, **kwargs):
        tic = time.time()

        explored_states = func(*args, **kwargs)

        toc = time.time()
        print(f'Compute time: {round(1000 * (toc - tic))}ms')
        return explored_states

    return inner


class State:
	def __init__(self, doctors, spells=[], pills=[], cost=0, parent=None):
		self.doctors = doctors
		self.spells = spells
		self.pills = pills 
		self.cost = cost
		self.parent = parent 

	def __eq__(self, object):
		if(isinstance(object, State)):
			return self.spells == object.spells and \
				self.pills == object.pills and \
				self.doctors == object.doctors

	def __str__(self):
		return f" doctors: {self.doctors}\n spells: {self.spells}\n pills: {self.pills}\n cost: {self.cost}\n"

	def __hash__(self) -> int:
		return hash(str(self))


def get_pair_input():
    a, b = map(int, input().strip().split())
    return a, b

n, m = get_pair_input()
c, k = get_pair_input()
grid = [[0 for i in range(m)] for j in range(n)]
row = n-1
for i in range(c):
    x, y = get_pair_input()
    grid[row-x][y] = 1      # add spells

for i in range(k):
    x, y = get_pair_input()
    grid[row-x][y] = 2      # add pills

d = int(input())
for i in range(d):
    x, y = get_pair_input()
    grid[row-x][y] = 3      # add walls

import numpy as np 
print(np.matrix(grid))
print()



x_dir = [ -1, 0, 1, 0]
y_dir = [ 0, 1, 0, -1]

explored = set()


def is_position_valid(x, y):
    """ check whether new position is valid"""
    if (x < 0 or y < 0 or x >= n or y >= m or grid[x][y] == 3):
        return False

    return True

def apply_event(state, grid, x, y):
    """ pick up consumables such as pills and spells """
    if(grid[x][y] == 1):
        if not (x, y) in state.spells:
            state.spells.append((x, y))

    elif(grid[x][y] == 2):
        if not (x,y) in state.pills:
            state.pills.append((x, y))
            state.doctors.append((0, 0))

def create_child_state(prev_state, x, y, j):
    """" generate child of the state """
    # create new state
    state = State(doctors=prev_state.doctors[:],
    spells=prev_state.spells[:], pills=prev_state.pills[:], 
    cost=prev_state.cost+1, parent=prev_state)

    # update doctor location
    state.doctors[j] = (x, y)

    apply_event(state, grid, x, y)
    
    return state

def goal_test(state):
    """" check if given state reached the goal """
    if len(state.spells) != c:
        return False 

    for doctor in state.doctors:
        if doctor != (0, m-1):
            return False

    return True 
    
def get_neighbors(doctors, j):
    """ get neighbors of a state in the game grid """
    neighbors = []
    for i in range(4):
        adjx = doctors[j][0] + x_dir[i]
        adjy = doctors[j][1] + y_dir[i]
        neighbors.append((adjx, adjy))
    return neighbors

def print_path(goal_state):
    print("path from source to goal: ")
    while goal_state:
        print(goal_state)
        goal_state = goal_state.parent

def DLS(grid, state ,maxDepth, explored):

    if goal_test(state):
        global goal_state 
        goal_state = state
        return True

    explored.add(state)
    if maxDepth <= 0 : return False

    doctors = state.doctors
    for j in range(len(doctors)):
        neighbors = get_neighbors(doctors, j)
        for adjx, adjy in neighbors:
            if (is_position_valid(adjx, adjy)):
                child_state = create_child_state(state, adjx, adjy, j)
                if(child_state not in explored):
                    if(DLS(grid, child_state ,maxDepth-1, explored)):
                        return True
    return False

@execution_time_wrapper
def IDS(grid, state, maxDepth):
    states_explored_sum = 0
    explored = set()
    apply_event(state, grid, n-1, 0)
    for depth in range(maxDepth):
        if (DLS(grid, state, depth, explored)):
            states_explored_sum += len(explored)
            return states_explored_sum
        explored = set()
    return states_explored_sum


start_state = State(doctors=[(n-1, 0)])


[[0 0 0 0]
 [1 0 0 0]
 [0 0 3 0]
 [0 2 1 0]]



In [56]:
explored_states = IDS(grid, start_state, n*m)
print("Unique States Seen: ", explored_states)
print_path(goal_state)

Compute time: 131ms
Unique States Seen:  1914
path from source to goal: 
 doctors: [(0, 3), (0, 3)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 11

 doctors: [(0, 3), (0, 2)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 10

 doctors: [(0, 3), (0, 1)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 9

 doctors: [(0, 3), (0, 0)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 8

 doctors: [(0, 3), (1, 0)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 cost: 7

 doctors: [(0, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 6

 doctors: [(1, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 5

 doctors: [(2, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 4

 doctors: [(3, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 3

 doctors: [(3, 2), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 cost: 2

 doctors: [(3, 1), (0, 0)]
 spells: []
 pills: [(3, 1)]
 cost: 1

 doctors: [(3, 0)]
 spells: []
 pills: []
 cost: 0



<table>
  <tr>
    <th></th>
    <th>Cost</th>
    <th>Unique States Seen</th>
    <th>Avg exec time (s)</th>
  </tr>
  <tr>
    <td>Test 1</td>
    <td>11</td>
    <td>1914</td>
    <td>0.108</td>
  </tr>
  <tr>
    <td>Test 2</td>
    <td>11</td>
    <td>319220</td>
    <td>17.5</td>
  </tr>
  <tr>
    <td>Test 3</td>
    <td>19</td>
    <td>507177</td>
    <td>29.2</td>
  </tr>
</table>

* Advantages:
    * optimal solution garanteed to be found
    * Far away targets are possible to be found earlier than BFS
    * space complexity is linear

* Disadvantages:
    * uninformed algorithm thus slow
    * so many states are seen multiple times

## A* Search

$ manhattan = |x_1 - y_1| + |x_2 - y_2| $

$ heuristic = \displaystyle\sum_{dr}{min(manhattan(dr, \{spells, target\}))} $

this heuristic is obviously admissible, because it calculates 
the min manhattan distance from doctors to spells and target and no path would ever be less than this.

it's also consistance because $ h(n) - h^*(n) < C $
it means that cost of each step are bigger equal than difference between heuristic calculated

In [59]:
import time 
import heapq

def execution_time_wrapper(func):
	def inner(*args, **kwargs):
		tic = time.time()

		result, explored_states = func(*args, **kwargs)

		toc = time.time()
		print(f'Compute time: {round(1000 * (toc - tic))}ms')
		return result, explored_states

	return inner

class State:
	def __init__(self, doctors, spells=[], pills=[], f=0, g=0, h=0, parent=None):
		self.doctors = doctors
		self.spells = spells
		self.pills = pills 
		self.f = f 
		self.g = g 
		self.h = h
		self.parent = parent 

	def __eq__(self, object):
		if(isinstance(object, State)):
			return self.spells == object.spells and \
				self.pills == object.pills and \
				self.doctors == object.doctors

	def __str__(self):
		return f" doctors: {self.doctors}\n spells: {self.spells}\n pills: {self.pills}\n f: {self.f} g: {self.g} h: {self.h}\n"

	def __hash__(self) -> int:
		return hash(f" doctors: {self.doctors}\n spells: {self.spells}\n pills: {self.pills}\n")

	def __lt__(self, obj):
		return self.f < obj.f


def get_pair_input():
	a, b = map(int, input().strip().split())
	return a, b

n, m = get_pair_input()
c, k = get_pair_input()
grid = [[0 for i in range(m)] for j in range(n)]
row = n-1
all_spells = []
for i in range(c):
	x, y = get_pair_input()
	grid[row-x][y] = 1      # add spells
	all_spells.append((row-x, y))

for i in range(k):
	x, y = get_pair_input()
	grid[row-x][y] = 2      # add pills

d = int(input())
for i in range(d):
	x, y = get_pair_input()
	grid[row-x][y] = 3      # add walls

import numpy as np 
print(np.matrix(grid))
print()



x_dir = [ -1, 0, 1, 0]
y_dir = [ 0, 1, 0, -1]

explored = set()

def manhattan(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

def heuristic(state):
	doctors = state.doctors
	spells_to_explore = [spell for spell in all_spells if (spell not in state.spells)]
	sum = 0
	for doctor in doctors:
		mini = manhattan(*doctor, *(0, m-1))
		for spell in spells_to_explore:
			mini = min(mini, manhattan(*doctor, *spell))
		sum += mini 

	return sum 


def is_position_valid(x, y):
    """ check whether new position is valid"""
    if (x < 0 or y < 0 or x >= n or y >= m or grid[x][y] == 3):
        return False

    return True

def apply_event(state, grid, x, y):
    """ pick up consumables such as pills and spells """
    if(grid[x][y] == 1):
        if not (x, y) in state.spells:
            state.spells.append((x, y))

    elif(grid[x][y] == 2):
        if not (x,y) in state.pills:
            state.pills.append((x, y))
            state.doctors.append((0, 0))

def create_child_state(prev_state, x, y, j):

	# create new state
	state = State(doctors=prev_state.doctors[:],
	spells=prev_state.spells[:], pills=prev_state.pills[:],
	parent=prev_state, g=prev_state.g+1)

	# update doctor location
	state.doctors[j] = (x, y)

	apply_event(state, grid, x, y)
	
	state.h = heuristic(state)
	state.f = state.h + state.g
	
	return state

def goal_test(state):
    """" check if given state reached the goal """
    if len(state.spells) != c:
        return False 

    for doctor in state.doctors:
        if doctor != (0, m-1):
            return False

    return True 
    
def get_neighbors(doctors, j):
    """ get neighbors of a state in the game grid """
    neighbors = []
    for i in range(4):
        adjx = doctors[j][0] + x_dir[i]
        adjy = doctors[j][1] + y_dir[i]
        neighbors.append((adjx, adjy))
    return neighbors

def print_path(goal_state):
    print("path from source to goal: ")
    while goal_state:
        print(goal_state)
        goal_state = goal_state.parent


@execution_time_wrapper
def astar(grid, start_state):

	openset = []
	closedset = set()
	heapq.heappush(openset, start_state)
	closedset.add(start_state)

	apply_event(start_state, grid, n-1, 0)

	while len(openset) > 0:
		state = heapq.heappop(openset)
		if goal_test(state):
			return state, len(closedset)

		doctors = state.doctors
		for j in range(len(doctors)):
			neighbors = get_neighbors(doctors, j)
			for adjx, adjy in neighbors:
				if (is_position_valid(adjx, adjy)):
					child_state = create_child_state(state, adjx, adjy, j)
					if child_state not in closedset:
						heapq.heappush(openset, child_state)
						closedset.add(child_state)


start_state = State(doctors=[(n-1, 0)])
start_state.f = start_state.h = heuristic(start_state)


[[0 0 0 0]
 [1 0 0 0]
 [0 0 3 0]
 [0 2 1 0]]



In [60]:
goal_state, explored_states = astar(grid, start_state)
print("Unique States Seen: ", explored_states)
print_path(goal_state)

Compute time: 10ms
Unique States Seen:  500
path from source to goal: 
 doctors: [(0, 3), (0, 3)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 f: 11 g: 11 h: 0

 doctors: [(0, 3), (0, 2)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 f: 11 g: 10 h: 1

 doctors: [(1, 3), (0, 2)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 f: 11 g: 9 h: 2

 doctors: [(1, 3), (0, 1)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 f: 11 g: 8 h: 3

 doctors: [(1, 3), (1, 1)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 f: 11 g: 7 h: 4

 doctors: [(1, 3), (1, 0)]
 spells: [(3, 2), (1, 0)]
 pills: [(3, 1)]
 f: 11 g: 6 h: 5

 doctors: [(1, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 f: 7 g: 5 h: 2

 doctors: [(2, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 f: 7 g: 4 h: 3

 doctors: [(3, 3), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 f: 7 g: 3 h: 4

 doctors: [(3, 2), (0, 0)]
 spells: [(3, 2)]
 pills: [(3, 1)]
 f: 7 g: 2 h: 5

 doctors: [(3, 1), (0, 0)]
 spells: []
 pills: [(3, 1)]
 f: 3 g: 1 h: 2

 doctors: 

<table>
  <tr>
    <th></th>
    <th>Cost</th>
    <th>Unique States Seen</th>
    <th>Avg exec time (s)</th>
  </tr>
  <tr>
    <td>Test 1</td>
    <td>11</td>
    <td>500</td>
    <td>0.010</td>
  </tr>
  <tr>
    <td>Test 2</td>
    <td>11</td>
    <td>8873</td>
    <td>0.172</td>
  </tr>
  <tr>
    <td>Test 3</td>
    <td>19</td>
    <td>25133</td>
    <td>0.538</td>
  </tr>
</table>

* Advantages:
    * will find optimal solution
    * informed algorithm thus faster than two previous algorithms
    * much less states are seen to reach the target
    * it's homing into the target



* Disadvatages:
    * hard to find its best heuristic

## Weighted A*

In [61]:
import time 
import heapq

def execution_time_wrapper(func):
	def inner(*args, **kwargs):
		tic = time.time()

		result, explored_states = func(*args, **kwargs)

		toc = time.time()
		print(f'Compute time: {round(1000 * (toc - tic))}ms')
		return result, explored_states

	return inner

class State:
	def __init__(self, doctors, spells=[], pills=[], f=0, g=0, h=0, parent=None):
		self.doctors = doctors
		self.spells = spells
		self.pills = pills 
		self.f = f 
		self.g = g 
		self.h = h
		self.parent = parent 

	def __eq__(self, object):
		if(isinstance(object, State)):
			return self.spells == object.spells and \
				self.pills == object.pills and \
				self.doctors == object.doctors

	def __str__(self):
		return f" doctors: {self.doctors}\n spells: {self.spells}\n pills: {self.pills}\n f: {self.f} g: {self.g} h: {self.h}\n"

	def __hash__(self) -> int:
		return hash(f" doctors: {self.doctors}\n spells: {self.spells}\n pills: {self.pills}\n")

	def __lt__(self, obj):
		return self.f < obj.f


def get_pair_input():
	a, b = map(int, input().strip().split())
	return a, b

n, m = get_pair_input()
c, k = get_pair_input()
grid = [[0 for i in range(m)] for j in range(n)]
row = n-1
all_spells = []
for i in range(c):
	x, y = get_pair_input()
	grid[row-x][y] = 1      # add spells
	all_spells.append((row-x, y))

for i in range(k):
	x, y = get_pair_input()
	grid[row-x][y] = 2      # add pills

d = int(input())
for i in range(d):
	x, y = get_pair_input()
	grid[row-x][y] = 3      # add walls

import numpy as np 
print(np.matrix(grid))
print()



x_dir = [ -1, 0, 1, 0]
y_dir = [ 0, 1, 0, -1]

explored = set()

def manhattan(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

def heuristic(state):
	doctors = state.doctors
	spells_to_explore = [spell for spell in all_spells if (spell not in state.spells)]
	sum = 0
	for doctor in doctors:
		mini = manhattan(*doctor, *(0, m-1))
		for spell in spells_to_explore:
			mini = min(mini, manhattan(*doctor, *spell))
		sum += mini 
        
	weight = 3
	return sum * weight


def is_position_valid(x, y):
    """ check whether new position is valid"""
    if (x < 0 or y < 0 or x >= n or y >= m or grid[x][y] == 3):
        return False

    return True

def apply_event(state, grid, x, y):
    """ pick up consumables such as pills and spells """
    if(grid[x][y] == 1):
        if not (x, y) in state.spells:
            state.spells.append((x, y))

    elif(grid[x][y] == 2):
        if not (x,y) in state.pills:
            state.pills.append((x, y))
            state.doctors.append((0, 0))

def create_child_state(prev_state, x, y, j):

	# create new state
	state = State(doctors=prev_state.doctors[:],
	spells=prev_state.spells[:], pills=prev_state.pills[:],
	parent=prev_state, g=prev_state.g+1)

	# update doctor location
	state.doctors[j] = (x, y)

	apply_event(state, grid, x, y)
	
	state.h = heuristic(state)
	state.f = state.h + state.g
	
	return state

def goal_test(state):
    """" check if given state reached the goal """
    if len(state.spells) != c:
        return False 

    for doctor in state.doctors:
        if doctor != (0, m-1):
            return False

    return True 
    
def get_neighbors(doctors, j):
    """ get neighbors of a state in the game grid """
    neighbors = []
    for i in range(4):
        adjx = doctors[j][0] + x_dir[i]
        adjy = doctors[j][1] + y_dir[i]
        neighbors.append((adjx, adjy))
    return neighbors

def print_path(goal_state):
    print("path from source to goal: ")
    while goal_state:
        print(goal_state)
        goal_state = goal_state.parent


@execution_time_wrapper
def astar(grid, start_state):

	openset = []
	closedset = set()
	heapq.heappush(openset, start_state)
	closedset.add(start_state)

	apply_event(start_state, grid, n-1, 0)

	while len(openset) > 0:
		state = heapq.heappop(openset)
		if goal_test(state):
			return state, len(closedset)

		doctors = state.doctors
		for j in range(len(doctors)):
			neighbors = get_neighbors(doctors, j)
			for adjx, adjy in neighbors:
				if (is_position_valid(adjx, adjy)):
					child_state = create_child_state(state, adjx, adjy, j)
					if child_state not in closedset:
						heapq.heappush(openset, child_state)
						closedset.add(child_state)


start_state = State(doctors=[(n-1, 0)])
start_state.f = start_state.h = heuristic(start_state)


[[0 0 0 0]
 [1 0 0 0]
 [0 0 3 0]
 [0 2 1 0]]



In [62]:
goal_state, explored_states = astar(grid, start_state)
print("Unique States Seen: ", explored_states)
print_path(goal_state)

Compute time: 11ms
Unique States Seen:  497
path from source to goal: 
 doctors: [(0, 3), (0, 3)]
 spells: [(1, 0), (3, 2)]
 pills: [(3, 1)]
 f: 11 g: 11 h: 0

 doctors: [(1, 3), (0, 3)]
 spells: [(1, 0), (3, 2)]
 pills: [(3, 1)]
 f: 13 g: 10 h: 3

 doctors: [(2, 3), (0, 3)]
 spells: [(1, 0), (3, 2)]
 pills: [(3, 1)]
 f: 15 g: 9 h: 6

 doctors: [(3, 3), (0, 3)]
 spells: [(1, 0), (3, 2)]
 pills: [(3, 1)]
 f: 17 g: 8 h: 9

 doctors: [(3, 2), (0, 3)]
 spells: [(1, 0), (3, 2)]
 pills: [(3, 1)]
 f: 19 g: 7 h: 12

 doctors: [(3, 1), (0, 3)]
 spells: [(1, 0)]
 pills: [(3, 1)]
 f: 9 g: 6 h: 3

 doctors: [(3, 1), (0, 2)]
 spells: [(1, 0)]
 pills: [(3, 1)]
 f: 11 g: 5 h: 6

 doctors: [(3, 1), (0, 1)]
 spells: [(1, 0)]
 pills: [(3, 1)]
 f: 13 g: 4 h: 9

 doctors: [(3, 1), (1, 1)]
 spells: [(1, 0)]
 pills: [(3, 1)]
 f: 15 g: 3 h: 12

 doctors: [(3, 1), (1, 0)]
 spells: [(1, 0)]
 pills: [(3, 1)]
 f: 17 g: 2 h: 15

 doctors: [(3, 1), (0, 0)]
 spells: []
 pills: [(3, 1)]
 f: 7 g: 1 h: 6

 doctors: [(

## $ \alpha = 2 $

<table>
  <tr>
    <th></th>
    <th>Cost</th>
    <th>Unique States Seen</th>
    <th>Avg exec time (s)</th>
  </tr>
  <tr>
    <td>Test 1</td>
    <td>11</td>
    <td>521</td>
    <td>0.010</td>
  </tr>
  <tr>
    <td>Test 2</td>
    <td>11</td>
    <td>3584</td>
    <td>0.060</td>
  </tr>
  <tr>
    <td>Test 3</td>
    <td>19</td>
    <td>9437</td>
    <td>0.170</td>
  </tr>
</table>

## $ \alpha = 3 $

<table>
  <tr>
    <th></th>
    <th>Cost</th>
    <th>Unique States Seen</th>
    <th>Avg exec time (s)</th>
  </tr>
  <tr>
    <td>Test 1</td>
    <td>11</td>
    <td>497</td>
    <td>0.009</td>
  </tr>
  <tr>
    <td>Test 2</td>
    <td>11</td>
    <td>3485</td>
    <td>0.054</td>
  </tr>
  <tr>
    <td>Test 3</td>
    <td>19</td>
    <td>5344</td>
    <td>0.092</td>
  </tr>
</table>

* Advatages:
    * much faster than A* itself
    * less states are explored
    

* Disadvatages:
    * may not find optimal solution